<a href="https://colab.research.google.com/github/WhiteFox-Lugh/Pratice-TextAnalysis/blob/master/exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 情報システム論実習　テキスト分析　課題1

逐次実行していけば所定の処理ができるようにしてあります。
（ただし、外部テキストファイルなどは適切な場所に配置してください）

# 0. 前処理

In [1]:
!pip install nltk
!pip install gensim

In [2]:
import nltk
import numpy as np
import pandas as pd
import re
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from google.colab import drive
from nltk.corpus import wordnet as wn
from nltk.metrics import jaccard_distance
from sklearn.feature_extraction.text import TfidfTransformer

In [3]:
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# drive のマウント（colaboratory 用）
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# テキスト読み込み
# テキストデータは適切なところに配置してください
text_name = ['canis', 'wolf', 'coyote', 'african_golden_wolf', 'dhole', 'african_wild_dog', 'red_fox', 'arctic_fox', 'fennec_fox', 'kitsune']
text_correction = {}
for i in range(len(text_name)):
  txt = f = open('drive/My Drive/Colab Notebooks/text' + str(i+1) + '.txt')
  data = f.read()
  name = text_name[i]
  text_correction[name] = data

In [6]:
# 読み込み確認
for txt in text_correction.values():
  print(txt)

Canis is a genus of the Caninae containing multiple extant species, such as wolves, dogs, coyotes and jackals. Species of this genus are distinguished by their moderate to large size, their massive, well-developed skulls and dentition, long legs, and comparatively short ears and tails.
The wolf (Canis lupus), also known as the gray wolf or grey wolf, is a large canine native to Eurasia and North America. More than thirty subspecies of Canis lupus have been recognized, and gray wolves, as colloquially understood, comprise non-domestic/feral subspecies. The wolf is the largest extant member of Canidae, males averaging 40 kg (88 lb) and females 37 kg (82 lb). Wolves measure 105–160 cm (41–63 in) in length and 80–85 cm (31–33 in) at shoulder height. The wolf is also distinguished from other Canis species by its less pointed ears and muzzle, as well as a shorter torso and a longer tail. The wolf is nonetheless related closely enough to smaller Canis species, such as the coyote and the golde

In [7]:
# preprocessing
# tokenize
text_tokenized = {}

def tokenize_text(text):
  text = re.sub('[.,()\[\]\-\%]', '', text)
  text = re.sub('[0-9]+', '', text)
  text = re.sub('–', '', text)
  ret = text.split()
  return ret

for i in range(len(text_name)):
  name = text_name[i]
  txt = text_correction[name]
  text_tokenized[name] = tokenize_text(txt)

In [8]:
# tokenize 確認
for txt in text_tokenized.values():
  print(txt)

['Canis', 'is', 'a', 'genus', 'of', 'the', 'Caninae', 'containing', 'multiple', 'extant', 'species', 'such', 'as', 'wolves', 'dogs', 'coyotes', 'and', 'jackals', 'Species', 'of', 'this', 'genus', 'are', 'distinguished', 'by', 'their', 'moderate', 'to', 'large', 'size', 'their', 'massive', 'welldeveloped', 'skulls', 'and', 'dentition', 'long', 'legs', 'and', 'comparatively', 'short', 'ears', 'and', 'tails']
['The', 'wolf', 'Canis', 'lupus', 'also', 'known', 'as', 'the', 'gray', 'wolf', 'or', 'grey', 'wolf', 'is', 'a', 'large', 'canine', 'native', 'to', 'Eurasia', 'and', 'North', 'America', 'More', 'than', 'thirty', 'subspecies', 'of', 'Canis', 'lupus', 'have', 'been', 'recognized', 'and', 'gray', 'wolves', 'as', 'colloquially', 'understood', 'comprise', 'nondomestic/feral', 'subspecies', 'The', 'wolf', 'is', 'the', 'largest', 'extant', 'member', 'of', 'Canidae', 'males', 'averaging', 'kg', 'lb', 'and', 'females', 'kg', 'lb', 'Wolves', 'measure', 'cm', 'in', 'in', 'length', 'and', 'cm', 

In [9]:
# lemmatize
def lemmatize_word(word_set):
  ret = []
  for word in word_set:
    word=word.lower()
    lemma = wn.morphy(word)
    if lemma is None:
      ret.append(word)
    else:
      ret.append(lemma)
  return ret

In [10]:
preprocessed_docs = {}

for i in range(len(text_name)):
  name = text_name[i]
  word_set = text_tokenized[name]
  preprocessed_docs[name] = lemmatize_word(word_set)

In [11]:
# preprocessing の check
for doc in preprocessed_docs.values():
  print(doc)

['canis', 'be', 'a', 'genus', 'of', 'the', 'caninae', 'contain', 'multiple', 'extant', 'species', 'such', 'as', 'wolf', 'dog', 'coyote', 'and', 'jackal', 'species', 'of', 'this', 'genus', 'are', 'distinguish', 'by', 'their', 'moderate', 'to', 'large', 'size', 'their', 'massive', 'welldeveloped', 'skull', 'and', 'dentition', 'long', 'legs', 'and', 'comparatively', 'short', 'ear', 'and', 'tails']
['the', 'wolf', 'canis', 'lupus', 'also', 'know', 'as', 'the', 'gray', 'wolf', 'or', 'grey', 'wolf', 'be', 'a', 'large', 'canine', 'native', 'to', 'eurasia', 'and', 'north', 'america', 'more', 'than', 'thirty', 'subspecies', 'of', 'canis', 'lupus', 'have', 'be', 'recognize', 'and', 'gray', 'wolf', 'as', 'colloquially', 'understand', 'comprise', 'nondomestic/feral', 'subspecies', 'the', 'wolf', 'be', 'the', 'large', 'extant', 'member', 'of', 'canidae', 'male', 'average', 'kg', 'lb', 'and', 'female', 'kg', 'lb', 'wolf', 'measure', 'cm', 'in', 'in', 'length', 'and', 'cm', 'in', 'at', 'shoulder', 'h

In [12]:
# 単語の set を構築する
docs_words_set = {}

for i in range(len(text_name)):
  name = text_name[i]
  words = preprocessed_docs[name]
  docs_words_set[name] = set(words)

In [13]:
# set の check
for word_set in docs_words_set.values():
  print(word_set)

{'moderate', 'such', 'are', 'welldeveloped', 'the', 'tails', 'short', 'caninae', 'dog', 'coyote', 'of', 'dentition', 'wolf', 'as', 'contain', 'skull', 'be', 'canis', 'size', 'this', 'multiple', 'ear', 'distinguish', 'jackal', 'large', 'genus', 'by', 'to', 'massive', 'their', 'and', 'comparatively', 'a', 'legs', 'extant', 'long', 'species'}
{'member', 'brown', 'female', 'muzzle', 'coyote', 'produce', 'as', 'eurasia', 'thirty', 'at', 'with', 'tail', 'colloquially', 'more', 'ear', 'may', 'cm', 'them', 'band', 'nonetheless', 'torso', 'male', 'nearly', 'lupus', 'fur', 'the', 'canine', 'of', 'arctic', 'wolf', 'be', 'enough', 'smaller', 'in', 'le', 'nondomestic/feral', 'north', 'measure', 'closely', 'have', 'shoulder', 'extant', 'white', 'species', 'other', 'although', 'golden', 'relate', 'all', 'point', 'well', 'subspecies', 'mottle', 'gray', 'jackal', 'large', 'from', 'native', 'black', 'height', 'such', 'than', 'lb', 'longer', 'usually', 'short', 'comprise', 'grey', 'know', 'also', 'it', '

In [14]:
# bag of words でベクトル化
def bow_vectorizer(docs):
  word2id = {}
  for doc in docs.values():
    for w in doc:
      if w not in word2id:
        word2id[w] = len(word2id)
        
  result_list = []
  for doc in docs.values():
    doc_vec = [0] * len(word2id)
    for w in doc:
      doc_vec[word2id[w]] += 1
    result_list.append(doc_vec)
  return result_list, word2id

bow_vec_doc, word2id_doc = bow_vectorizer(preprocessed_docs)

In [15]:
# bow の check
for i in range(len(text_name)):
  name = text_name[i]
  vector = bow_vec_doc[i]
  print("vector(" + name + ") -> " + str(vector))

vector(canis) -> [1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [16]:
word2id_doc.items()

dict_items([('canis', 0), ('be', 1), ('a', 2), ('genus', 3), ('of', 4), ('the', 5), ('caninae', 6), ('contain', 7), ('multiple', 8), ('extant', 9), ('species', 10), ('such', 11), ('as', 12), ('wolf', 13), ('dog', 14), ('coyote', 15), ('and', 16), ('jackal', 17), ('this', 18), ('are', 19), ('distinguish', 20), ('by', 21), ('their', 22), ('moderate', 23), ('to', 24), ('large', 25), ('size', 26), ('massive', 27), ('welldeveloped', 28), ('skull', 29), ('dentition', 30), ('long', 31), ('legs', 32), ('comparatively', 33), ('short', 34), ('ear', 35), ('tails', 36), ('lupus', 37), ('also', 38), ('know', 39), ('gray', 40), ('or', 41), ('grey', 42), ('canine', 43), ('native', 44), ('eurasia', 45), ('north', 46), ('america', 47), ('more', 48), ('than', 49), ('thirty', 50), ('subspecies', 51), ('have', 52), ('recognize', 53), ('colloquially', 54), ('understand', 55), ('comprise', 56), ('nondomestic/feral', 57), ('member', 58), ('canidae', 59), ('male', 60), ('average', 61), ('kg', 62), ('lb', 63),

# 1. ドキュメントの類似度の計算

## 1-1. 単語の集合(set)ベース

### Jaccard 係数

In [17]:
# Jaccard 係数
jaccard_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    set_i = docs_words_set[name_i]
    set_j = docs_words_set[name_j]
    jaccard = 1 - jaccard_distance(set_i, set_j)
    name_ij = "Jaccard(" + name_i + ", " + name_j + ")"
    jaccard_res[name_ij] = jaccard

In [18]:
# Jaccard 係数の降順に出力
jaccard_sim_sorted = sorted(jaccard_res.items(), key = lambda x : -x[1])
for key, value in jaccard_sim_sorted:
  print(key + " -> " + str(value))

Jaccard(wolf, coyote) -> 0.24561403508771928
Jaccard(coyote, dhole) -> 0.2075471698113207
Jaccard(african_golden_wolf, african_wild_dog) -> 0.19548872180451127
Jaccard(coyote, african_golden_wolf) -> 0.18487394957983194
Jaccard(canis, african_wild_dog) -> 0.18478260869565222
Jaccard(canis, wolf) -> 0.17924528301886788
Jaccard(coyote, african_wild_dog) -> 0.17757009345794394
Jaccard(wolf, african_wild_dog) -> 0.17647058823529416
Jaccard(canis, coyote) -> 0.16666666666666663
Jaccard(arctic_fox, fennec_fox) -> 0.16666666666666663
Jaccard(african_wild_dog, red_fox) -> 0.16521739130434787
Jaccard(african_golden_wolf, red_fox) -> 0.1640625
Jaccard(wolf, arctic_fox) -> 0.16176470588235292
Jaccard(dhole, african_wild_dog) -> 0.15873015873015872
Jaccard(african_wild_dog, arctic_fox) -> 0.14516129032258063
Jaccard(red_fox, arctic_fox) -> 0.13793103448275867
Jaccard(coyote, red_fox) -> 0.13725490196078427
Jaccard(wolf, african_golden_wolf) -> 0.13636363636363635
Jaccard(red_fox, fennec_fox) -> 0.

### Dice 係数

In [19]:
# Dice 係数
def dice_sim(set_a, set_b):
  num_intersect = len(set.intersection(set_a, set_b))
  s = len(set_a) + len(set_b)
  ret = (2 * num_intersect / s) if s != 0 else 1.0
  return ret

In [20]:
# Dice 係数計算
dice_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    set_i = docs_words_set[name_i]
    set_j = docs_words_set[name_j]
    dice = dice_sim(set_i, set_j)
    name_ij = "Dice(" + name_i + ", " + name_j + ")"
    dice_res[name_ij] = dice

In [21]:
# Dice 係数の降順に出力
dice_sim_sorted = sorted(dice_res.items(), key = lambda x : -x[1])
for key, value in dice_sim_sorted:
  print(key + " -> " + str(value))

Dice(wolf, coyote) -> 0.39436619718309857
Dice(coyote, dhole) -> 0.34375
Dice(african_golden_wolf, african_wild_dog) -> 0.3270440251572327
Dice(coyote, african_golden_wolf) -> 0.3120567375886525
Dice(canis, african_wild_dog) -> 0.3119266055045872
Dice(canis, wolf) -> 0.304
Dice(coyote, african_wild_dog) -> 0.30158730158730157
Dice(wolf, african_wild_dog) -> 0.3
Dice(canis, coyote) -> 0.2857142857142857
Dice(arctic_fox, fennec_fox) -> 0.2857142857142857
Dice(african_wild_dog, red_fox) -> 0.2835820895522388
Dice(african_golden_wolf, red_fox) -> 0.28187919463087246
Dice(wolf, arctic_fox) -> 0.27848101265822783
Dice(dhole, african_wild_dog) -> 0.273972602739726
Dice(african_wild_dog, arctic_fox) -> 0.2535211267605634
Dice(red_fox, arctic_fox) -> 0.24242424242424243
Dice(coyote, red_fox) -> 0.2413793103448276
Dice(wolf, african_golden_wolf) -> 0.24
Dice(red_fox, fennec_fox) -> 0.24
Dice(wolf, fennec_fox) -> 0.2251655629139073
Dice(african_golden_wolf, dhole) -> 0.2236024844720497
Dice(afric

### Szymkiewicz-Simpson 係数

In [22]:
# Szymkiewicz-Simpson 係数
def simpson_sim(set_a, set_b):
  num_intersect = len(set.intersection(set_a, set_b))
  m = min(len(set_a), len(set_b))
  ret = num_intersect / m if m != 0 else 1.0
  return ret

In [23]:
# Simpson 係数計算
simpson_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    set_i = docs_words_set[name_i]
    set_j = docs_words_set[name_j]
    simpson = simpson_sim(set_i, set_j)
    name_ij = "Simpson(" + name_i + ", " + name_j + ")"
    simpson_res[name_ij] = simpson

In [24]:
# Simpson 係数の降順に出力
simpson_sim_sorted = sorted(simpson_res.items(), key = lambda x : -x[1])
for key, value in simpson_sim_sorted:
  print(key + " -> " + str(value))

Simpson(wolf, coyote) -> 0.5185185185185185
Simpson(canis, wolf) -> 0.5135135135135135
Simpson(canis, african_wild_dog) -> 0.4594594594594595
Simpson(coyote, african_golden_wolf) -> 0.4074074074074074
Simpson(coyote, dhole) -> 0.4074074074074074
Simpson(african_golden_wolf, african_wild_dog) -> 0.3611111111111111
Simpson(coyote, african_wild_dog) -> 0.35185185185185186
Simpson(canis, coyote) -> 0.35135135135135137
Simpson(canis, african_golden_wolf) -> 0.35135135135135137
Simpson(african_golden_wolf, red_fox) -> 0.3387096774193548
Simpson(wolf, african_wild_dog) -> 0.3333333333333333
Simpson(canis, dhole) -> 0.32432432432432434
Simpson(wolf, arctic_fox) -> 0.3142857142857143
Simpson(african_wild_dog, red_fox) -> 0.3064516129032258
Simpson(arctic_fox, fennec_fox) -> 0.30158730158730157
Simpson(canis, kitsune) -> 0.2972972972972973
Simpson(dhole, african_wild_dog) -> 0.2777777777777778
Simpson(arctic_fox, kitsune) -> 0.2714285714285714
Simpson(canis, fennec_fox) -> 0.2702702702702703
Sim

## 1-2. ベクトル空間モデルにおける類似度計算

### Bag of Words と Cosine 類似度

In [25]:
# cosine 類似度
def cos_sim(v_a, v_b):
  norm2_a = np.linalg.norm(v_a)
  norm2_b = np.linalg.norm(v_b)
  in_prod = np.array(v_a).dot(np.array(v_b))
  ret = (in_prod / (norm2_a * norm2_b)) if norm2_a * norm2_b != 0 else 1.0
  return ret

In [26]:
# cosine 類似度の計算
cos_sim_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    vec_i = bow_vec_doc[i]
    vec_j = bow_vec_doc[j]
    cos = cos_sim(vec_i, vec_j)
    name_ij = "Cosine(" + name_i + ", " + name_j + ")"
    cos_sim_res[name_ij] = cos

In [27]:
# Bag of Words -> Cosine 類似度の降順に出力
cos_sim_sorted = sorted(cos_sim_res.items(), key = lambda x : -x[1])
for key, value in cos_sim_sorted:
  print(key + " -> " + str(value))

Cosine(wolf, coyote) -> 0.7635241616868097
Cosine(wolf, african_golden_wolf) -> 0.7231791933019218
Cosine(coyote, african_golden_wolf) -> 0.7070130569740437
Cosine(african_golden_wolf, african_wild_dog) -> 0.6634113453812164
Cosine(african_wild_dog, red_fox) -> 0.6580027030734056
Cosine(coyote, red_fox) -> 0.6560794100967968
Cosine(coyote, african_wild_dog) -> 0.6340787019713607
Cosine(african_golden_wolf, red_fox) -> 0.6316405573900559
Cosine(red_fox, arctic_fox) -> 0.6314497500413865
Cosine(wolf, african_wild_dog) -> 0.6274323114889218
Cosine(arctic_fox, fennec_fox) -> 0.6106359356250138
Cosine(coyote, dhole) -> 0.5992771223631909
Cosine(arctic_fox, kitsune) -> 0.5976112869643463
Cosine(red_fox, fennec_fox) -> 0.5949613152809715
Cosine(african_golden_wolf, dhole) -> 0.5771931941422757
Cosine(dhole, african_wild_dog) -> 0.5702418861501779
Cosine(african_golden_wolf, kitsune) -> 0.5580312500603
Cosine(wolf, arctic_fox) -> 0.550312722431504
Cosine(wolf, red_fox) -> 0.5454193633171008
Co

### TF-IDF と Cosine 類似度

In [28]:
# TF-IDF
# bow を ndarray の 2次元配列に変換
counts = np.array(bow_vec_doc)

# transformer : idf は 1 を加えて smoothing する
transformer = TfidfTransformer(smooth_idf=False)

# TF-IDF ベクトルを得る
tf_idf = transformer.fit_transform(counts)
tf_idf_array = tf_idf.toarray()
print(tf_idf_array)

[[0.093682   0.06200715 0.06854026 ... 0.         0.         0.        ]
 [0.16115246 0.18666404 0.11790346 ... 0.         0.         0.        ]
 [0.06361943 0.16843619 0.04654568 ... 0.         0.         0.        ]
 ...
 [0.         0.13511906 0.07467763 ... 0.         0.         0.        ]
 [0.         0.12129654 0.0335191  ... 0.         0.         0.        ]
 [0.         0.04435863 0.09806456 ... 0.07324908 0.07324908 0.07324908]]


In [29]:
# TF-IDF での cosine 類似度の計算
cos_sim_tfidf_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    name_i = text_name[i]
    name_j = text_name[j]
    vec_i = tf_idf_array[i]
    vec_j = tf_idf_array[j]
    cos = cos_sim(vec_i, vec_j)
    name_ij = "TF-IDF_Cosine(" + name_i + ", " + name_j + ")"
    cos_sim_tfidf_res[name_ij] = cos

In [30]:
# TF-IDF -> Cosine 類似度の降順に出力
cos_sim_tfidf_sorted = sorted(cos_sim_tfidf_res.items(), key = lambda x : -x[1])

for key, value in cos_sim_tfidf_sorted:
  print(key + " -> " + str(value))

TF-IDF_Cosine(wolf, coyote) -> 0.45659482854645833
TF-IDF_Cosine(coyote, african_golden_wolf) -> 0.36119930063872907
TF-IDF_Cosine(wolf, african_golden_wolf) -> 0.3410146592985079
TF-IDF_Cosine(african_golden_wolf, african_wild_dog) -> 0.3245739181371838
TF-IDF_Cosine(african_golden_wolf, red_fox) -> 0.30054083491636424
TF-IDF_Cosine(red_fox, arctic_fox) -> 0.2953074762014565
TF-IDF_Cosine(african_wild_dog, red_fox) -> 0.2890493288479826
TF-IDF_Cosine(coyote, red_fox) -> 0.28103075868253496
TF-IDF_Cosine(dhole, african_wild_dog) -> 0.2782160488367521
TF-IDF_Cosine(coyote, african_wild_dog) -> 0.27239868196990286
TF-IDF_Cosine(wolf, african_wild_dog) -> 0.27084130014545277
TF-IDF_Cosine(wolf, arctic_fox) -> 0.261911285928645
TF-IDF_Cosine(coyote, dhole) -> 0.25710105314102244
TF-IDF_Cosine(arctic_fox, kitsune) -> 0.2564310852703539
TF-IDF_Cosine(canis, wolf) -> 0.25541490913542253
TF-IDF_Cosine(arctic_fox, fennec_fox) -> 0.25490693871030123
TF-IDF_Cosine(red_fox, fennec_fox) -> 0.225697

## 1-3. 単語の分散表現とドキュメントの類似度

### Doc2Vec

In [31]:
# Doc2Vec の入力
labeled_sentence = [TaggedDocument(words = text_tokenized[n], tags = [n]) 
                    for n in text_name]

print(labeled_sentence)

[TaggedDocument(words=['Canis', 'is', 'a', 'genus', 'of', 'the', 'Caninae', 'containing', 'multiple', 'extant', 'species', 'such', 'as', 'wolves', 'dogs', 'coyotes', 'and', 'jackals', 'Species', 'of', 'this', 'genus', 'are', 'distinguished', 'by', 'their', 'moderate', 'to', 'large', 'size', 'their', 'massive', 'welldeveloped', 'skulls', 'and', 'dentition', 'long', 'legs', 'and', 'comparatively', 'short', 'ears', 'and', 'tails'], tags=['canis']), TaggedDocument(words=['The', 'wolf', 'Canis', 'lupus', 'also', 'known', 'as', 'the', 'gray', 'wolf', 'or', 'grey', 'wolf', 'is', 'a', 'large', 'canine', 'native', 'to', 'Eurasia', 'and', 'North', 'America', 'More', 'than', 'thirty', 'subspecies', 'of', 'Canis', 'lupus', 'have', 'been', 'recognized', 'and', 'gray', 'wolves', 'as', 'colloquially', 'understood', 'comprise', 'nondomestic/feral', 'subspecies', 'The', 'wolf', 'is', 'the', 'largest', 'extant', 'member', 'of', 'Canidae', 'males', 'averaging', 'kg', 'lb', 'and', 'females', 'kg', 'lb', '

In [32]:
# モデルの定義
# dm : 1 なら PV-DM を使い、0 なら PV-DBOW を使う
# vector_size : 特徴ベクトルの次元数
# windows : 前後何単語とるか
# min_count : total frequency がこれ未満なら単語を無視する（今回は無視しない）
# alpha : 学習率
model = Doc2Vec(documents = labeled_sentence, dm = 1, vector_size = 100,
                windows = 5, min_count = 1, alpha = 0.0015)

In [33]:
# 文書の類似度を取得
doc2vec_sim_res = {}
for i in range(len(text_name)):
  for j in range(i + 1, len(text_name)):
    tag_i = text_name[i]
    tag_j = text_name[j]
    sim = model.docvecs.similarity(tag_i, tag_j)
    tag_ij = "Doc2Vec_Cosine(" + tag_i + ", " + tag_j + ")"
    doc2vec_sim_res[tag_ij] = sim

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [34]:
# Doc2Vec -> Cosine 類似度の降順に出力
doc2vec_sim_res_sorted = sorted(doc2vec_sim_res.items(), key = lambda x : -x[1])

for key, value in doc2vec_sim_res_sorted:
  print(key + " -> " + str(value))

Doc2Vec_Cosine(coyote, dhole) -> 0.2843592
Doc2Vec_Cosine(arctic_fox, kitsune) -> 0.26412058
Doc2Vec_Cosine(african_golden_wolf, dhole) -> 0.18594341
Doc2Vec_Cosine(dhole, red_fox) -> 0.16068977
Doc2Vec_Cosine(dhole, fennec_fox) -> 0.1298356
Doc2Vec_Cosine(dhole, african_wild_dog) -> 0.126938
Doc2Vec_Cosine(canis, african_wild_dog) -> 0.11929832
Doc2Vec_Cosine(red_fox, fennec_fox) -> 0.11473014
Doc2Vec_Cosine(coyote, african_golden_wolf) -> 0.106517196
Doc2Vec_Cosine(african_wild_dog, kitsune) -> 0.10125299
Doc2Vec_Cosine(canis, arctic_fox) -> 0.09719852
Doc2Vec_Cosine(african_wild_dog, fennec_fox) -> 0.09678839
Doc2Vec_Cosine(coyote, african_wild_dog) -> 0.08316032
Doc2Vec_Cosine(canis, wolf) -> 0.08232581
Doc2Vec_Cosine(canis, kitsune) -> 0.07931926
Doc2Vec_Cosine(african_golden_wolf, kitsune) -> 0.063464604
Doc2Vec_Cosine(african_golden_wolf, red_fox) -> 0.06115647
Doc2Vec_Cosine(coyote, arctic_fox) -> 0.060937066
Doc2Vec_Cosine(coyote, kitsune) -> 0.05289017
Doc2Vec_Cosine(arctic_f

# 2. 考察と感想

GitHub Repository: https://github.com/WhiteFox-Lugh/Pratice-TextAnalysis

今回はドキュメントの類似度を計算する方法として以下の文書表現手法と距離尺度を用いた。

- 文書を単語の集合で表現する
  - Jaccard 係数、Dice 係数、 Simpson 係数
- 文書をベクトルで表現する
  - Bag of Words -> Cosine 類似度
  - TF-IDF -> Cosine 類似度
- 文書を分散表現で表現する（機械学習的手法）
  - doc2vec

入力ドキュメントは、GitHub の Repository に text1.txt ～ text10.txt として置いてある。

text の引用元を以下に示す。全て最初の概要部分1パラグラフ分をとってきている（注釈番号は抜いた）。

* text1.txt  : https://en.wikipedia.org/wiki/Canis
* text2.txt  : https://en.wikipedia.org/wiki/Wolf
* text3.txt  : https://en.wikipedia.org/wiki/Coyote
* text4.txt  : https://en.wikipedia.org/wiki/African_golden_wolf
* text5.txt  : https://en.wikipedia.org/wiki/Dhole
* text6.txt  : https://en.wikipedia.org/wiki/African_wild_dog
* text7.txt  : https://en.wikipedia.org/wiki/Red_fox
* text8.txt  : https://en.wikipedia.org/wiki/Arctic_fox
* text9.txt  : https://en.wikipedia.org/wiki/Fennec_fox
* text10.txt : https://en.wikipedia.org/wiki/Kitsune

各文書表現手法と距離尺度における実行結果と考察を記述していく。

はじめに、文書を単語の集合で表現する手法についてみていく。

Jaccard 係数による比較では、二つの文章に共通する単語と和集合で定義を行っている。Dice 係数と Simpson 係数では、二つの文書に共通して登場する単語と、それぞれの文書の単語数だけを見ている。
利点としては二つの文書内の単語の種類の総数と、共通する単語だけを列挙すればいいので非常に単純な実装でできる点である。

Jaccard 係数での文書類似度比較の上位5ペアは以下のようになった。
- Jaccard(wolf, coyote) -> 0.24561403508771928
- Jaccard(coyote, dhole) -> 0.2075471698113207
- Jaccard(african_golden_wolf, african_wild_dog) -> 0.19548872180451127
- Jaccard(coyote, african_golden_wolf) -> 0.18487394957983194
- Jaccard(canis, african_wild_dog) -> 0.18478260869565222

Dice 係数での文書類似度の比較の上位5ペアは以下のようになった。
- Dice(wolf, coyote) -> 0.39436619718309857
- Dice(coyote, dhole) -> 0.34375
- Dice(african_golden_wolf, african_wild_dog) -> 0.3270440251572327
- Dice(coyote, african_golden_wolf) -> 0.3120567375886525
- Dice(canis, african_wild_dog) -> 0.3119266055045872

Simpson 係数での文書の類似度の比較の上位5ペアは以下のようになった。
- Simpson(wolf, coyote) -> 0.5185185185185185
- Simpson(canis, wolf) -> 0.5135135135135135
- Simpson(canis, african_wild_dog) -> 0.4594594594594595
- Simpson(coyote, african_golden_wolf) -> 0.4074074074074074
- Simpson(coyote, dhole) -> 0.4074074074074074

いずれにおいても、オオカミに関する文書同士の類似度が高くなった。一方、text7.txt ～ text10.txt ではキツネに関する文書を用意したがいずれにおいても類似度の上位には入ってこなかった。

単純な手法で文書の類似度が比較できるが、単語が出現するか否かしか考えていない。そのため、短い文書に対してはよい手法であるといえそうだが、文書のコレクションサイズ（文書数）が大きくなったり、一つ一つのドキュメントの長さが長くなってくると、共通する単語数がそもそも少なくなってくると予想できるため、類似度の値が小さくなり、比較が困難になると考えられる。また、文脈や単語のつながり、Los Angels といったように二語で一つの単語を構成する単語については何も考えられていないのも問題である。

次に、Bag of Words と Cosine 類似度、および TF-IDF と Cosine 類似度による文書の類似度の比較について考察する。

Bag of Words と Cosine 類似度による文書の類似度ランキングの上位5ペアは以下のようになった。

- Cosine(wolf, coyote) -> 0.7635241616868097
- Cosine(wolf, african_golden_wolf) -> 0.7231791933019218
- Cosine(coyote, african_golden_wolf) -> 0.7070130569740437
- Cosine(african_golden_wolf, african_wild_dog) -> 0.6634113453812164
- Cosine(african_wild_dog, red_fox) -> 0.6580027030734056

TF-IDF と Cosine 類似度による文書の類似度ランキングの上位5ペアは以下のようになった。

- TF-IDF_Cosine(wolf, coyote) -> 0.45659482854645833
- TF-IDF_Cosine(coyote, african_golden_wolf) -> 0.36119930063872907
- TF-IDF_Cosine(wolf, african_golden_wolf) -> 0.3410146592985079
- TF-IDF_Cosine(african_golden_wolf, african_wild_dog) -> 0.3245739181371838
- TF-IDF_Cosine(african_golden_wolf, red_fox) -> 0.30054083491636424

先ほどの単語の集合ベースとはまた少し異なった結果が得られたことがわかる。

Bag of Words では、集合ベースの方法と比較すると、単語の出現頻度を考慮できている点ではよい手法であると考えられる。また、TF-IDF では文書内での単語の出現頻度、および文書間での単語の出現頻度を同時に考慮できている。コサイン類似度の大きさを見れば、文書ベクトルがどれくらい似ているかもわかりやすい。しかし、単語が出現するか否か、出現頻度しかみていないので文脈については何も考慮できていないところは問題である。

最後に、機械学習的手法（Doc2Vec）による文書の類似度比較について考察する。

Doc2Vec による文書の類似度のランキング上位5ペアは以下のようになった。

- Doc2Vec_Cosine(coyote, dhole) -> 0.2843592
- Doc2Vec_Cosine(arctic_fox, kitsune) -> 0.26412058
- Doc2Vec_Cosine(african_golden_wolf, dhole) -> 0.18594341
- Doc2Vec_Cosine(dhole, red_fox) -> 0.16068977
- Doc2Vec_Cosine(dhole, fennec_fox) -> 0.1298356

集合ベースやベクトル空間モデルでの文章と全く異なった結果が得られた。特に、オオカミ系の文書とキツネ系の文書の類似度が高いという結果が上位に来ているのが大きな違いである。

また、類似度の値についてみてみると、集合ベースの距離尺度と Bag of Words から Cosine 類似度、TF-IDF から Cosine 類似度を算出する方法は0以上1以下の値をとるが、Doc2Vec から Cosine 類似度は -1 以上 1 以下の値をとる。Bag of Words と TF-IDF ではベクトルの各成分はすべて正なので、Cosine 類似度の値は負にならない。

Doc2Vec は、機械学習的な手法であるため、ほかの二つの手法よりも膨大なサイズの文書コレクション、文書長のドキュメント対して有効な手法と考えられる。この手法では、今回実装したほかの手法では考慮できていない単語の前後の文脈を考慮することができている。

## 感想

単語の適切な前処理について、正規表現を用いていらない情報を除去するのが思いのほか難しかった。今回は英語であったが、自分の研究では日本語のテキスト分析を行う機会も出てくると思うので、そちらに応用できるように前処理方法についてはもう少ししっかり勉強していきたい。

ドキュメントの類似度については、単語の set を構築する方法と、ベクトル空間モデルによる手法は以前に勉強して実装したことがあるので、今回は、Word2Vec や Doc2Vec による手法を理解することに力を入れた。ライブラリの使い方に慣れておらず、公式ドキュメントとそこに書かれているサンプルコードを頼りに実装したが、なかなか難しかった。